# Prediksi Gagal Jantung

## Import Data

In [16]:
# Import library pengolahan struktur data
import pandas as pd

# Import library pengolahan angka
import numpy as np

In [25]:
## Bikin Fungsinya
def import_data(filename):
    """
    Fungsi import data dan hapus duplikat
    :param filename: string nama file.csv
    :return df: <pandas data frame> sampel data
    """
    
    # print nama file
    print("Source data : ",filename)
    
    # read data
    df = pd.read_csv(filename)
    print("Data asli : ",df.shape,"(observasi, kolom)")
    
    # hapus duplikat
    df = df.drop_duplicates()
    print("Data setelah drop dulikat :",df.shape,"(observasi, kolom)")
    
    #jangan lupa return
    return df
    
# state lokasi file
data_src = "/home/jovyan/work/project/heart_failure_clinical_records_dataset.csv"

# jalankan import data
heartfail_df = import_data(filename = data_src)

Source data :  /home/jovyan/work/project/heart_failure_clinical_records_dataset.csv
Data asli :  (299, 13) (observasi, kolom)
Data setelah drop dulikat : (299, 13) (observasi, kolom)


In [26]:
heartfail_df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## Data Preprocessing

In [30]:
# buat output data
output_data = heartfail_df['DEATH_EVENT']

# buat input data
input_data = heartfail_df.drop('DEATH_EVENT', axis = 1)

def extract_input_output